# Traveling Salesman Problem (TSP) - Genetic and Greedy Algorithms

This notebook presents comprehensive implementations of both Genetic and Greedy algorithms to solve the **Traveling Salesman Problem (TSP)**. The TSP is a classic optimization problem aiming to find the shortest possible route that visits a set of cities exactly once and returns to the origin city.

### Table of Contents
1. [Introduction](#Introduction)
2. [Importing Libraries](#Importing-Libraries)
3. [Utility Functions](#Utility-Functions)
    - [Distance Matrix Creation](#Distance-Matrix-Creation)
    - [TSP Cost Calculation](#TSP-Cost-Calculation)
    - [Adaptive Population Size](#Adaptive-Population-Size)
    - [Population Initialization](#Population-Initialization)
    - [Selection Mechanism](#Selection-Mechanism)
    - [Crossover and Mutation](#Crossover-and-Mutation)
4. [Genetic Algorithm Implementation](#Genetic-Algorithm-Implementation)
5. [Greedy Algorithm Implementation](#Greedy-Algorithm-Implementation)
6. [Running Experiments](#Running-Experiments)
7. [Results and Analysis](#Results-and-Analysis)
8. [Plotting Progressions](#Plotting-Progressions)

## Introduction

The **Traveling Salesman Problem (TSP)** is a well-known algorithmic problem in the field of computer science and operations research. It focuses on optimization. In this problem, a salesman is given a list of cities and must determine the shortest possible route that visits each city exactly once and returns to the origin city.

This notebook implements two approaches to solve the TSP:

1. **Genetic Algorithm (GA):** An evolutionary algorithm inspired by natural selection, which iteratively evolves a population of candidate solutions.
2. **Greedy Algorithm:** A heuristic-based approach that builds a solution step-by-step, always choosing the locally optimal choice at each step.

By comparing these two methods, we can analyze their effectiveness, efficiency, and applicability to different datasets.

## Importing Libraries

We begin by importing the necessary libraries required for data manipulation, mathematical computations, geographic distance calculations, and logging. These libraries include:

- `logging`: For logging informational and error messages.
- `pandas`: For data manipulation and analysis.
- `numpy`: For numerical computations.
- `geopy`: Specifically, `geodesic` for calculating geodesic distances between coordinates.
- `itertools`: For efficient looping, particularly the `combinations` function.
- `csv`: For reading and writing CSV files.
- `time`: For measuring execution time.
- `matplotlib`: For plotting results.

In [1]:
import logging
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from itertools import combinations
import csv  # For saving results
import time  # For measuring elapsed time
import matplotlib.pyplot as plt  # For plotting

# Configure logging
logging.basicConfig(level=logging.INFO)

# Set a seed for reproducibility
SEED = 42
rng = np.random.default_rng(np.random.PCG64(SEED))

## Utility Functions

This section encompasses a series of utility functions essential for setting up and running both the Genetic and Greedy algorithms. These functions handle tasks such as creating distance matrices, calculating tour costs, initializing populations, and performing selection, crossover, and mutation operations.

### Distance Matrix Creation

The `create_distance_matrix` function computes a symmetric matrix representing the geodesic distances between each pair of cities. This matrix serves as the foundational data structure for evaluating tour costs in both algorithms.

**Parameters:**
- `cities` (`pd.DataFrame`): A DataFrame containing city information with columns `'name'`, `'lat'`, and `'lon'`.

**Returns:**
- `np.ndarray`: A symmetric matrix where the entry at position `(i, j)` represents the distance between city `i` and city `j` in kilometers.

In [2]:
def create_distance_matrix(cities):
    """
    Create a distance matrix for a list of cities based on geodesic distance.

    Parameters:
        cities (pd.DataFrame): DataFrame with columns 'name', 'lat', 'lon'.

    Returns:
        np.ndarray: A symmetric matrix where entry (i, j) is the distance between city i and city j.
    """
    dist_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(index=True), 2):
        distance = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km
        dist_matrix[c1.Index, c2.Index] = distance
        dist_matrix[c2.Index, c1.Index] = distance
    return dist_matrix

### TSP Cost Calculation

The `tsp_cost` function calculates the total distance of a given TSP tour based on the distance matrix. It ensures that the tour is a cycle by checking if the first and last cities are the same.

**Parameters:**
- `tsp` (`list`): A list of integers representing the tour, where each integer is an index of a city.
- `dist_matrix` (`np.ndarray`): The distance matrix for the cities.

**Returns:**
- `float`: The total cost (distance) of the tour in kilometers.

In [3]:
def tsp_cost(tsp, dist_matrix):
    """
    Calculate the total distance (cost) of a given TSP tour.

    Parameters:
        tsp (list): A list of integers representing a tour, where each integer is an index of a city.
        dist_matrix (np.ndarray): The distance matrix for the cities.

    Returns:
        float: The total cost (distance) of the tour.
    """
    if tsp[0] != tsp[-1]:  # Ensure the tour is a cycle
        tsp = tsp + [tsp[0]]
    tot_cost = sum(dist_matrix[tsp[i], tsp[i + 1]] for i in range(len(tsp) - 1))
    return tot_cost

### Adaptive Population Size

The `adaptive_population` function determines an optimal population size based on the number of cities.

**Parameters:**
- `cities_count` (`int`): The number of cities in the dataset.

**Returns:**
- `int`: An optimal population size.

In [4]:
def adaptive_population(cities_count):
    """
    Determine an adaptive population size based on the number of cities.

    Parameters:
        cities_count (int): The number of cities in the dataset.

    Returns:
        int: An optimal population size.
    """
    if cities_count < 50:
        return 100
    elif cities_count < 200:
        return 200
    else:
        return 300

### Population Initialization

The `initialize_population` function generates an initial population of random TSP tours. Each tour is a random permutation of city indices, ensuring diversity in the initial population.

**Parameters:**
- `cities_count` (`int`): The number of cities in the dataset.
- `pop_size` (`int`): The number of tours to generate.

**Returns:**
- `list of lists`: A population of TSP tours. Each tour is a list of city indices representing a cycle (starts and ends at the same city).

In [5]:
def initialize_population(cities_count, pop_size):
    """
    Generate an initial population of random TSP tours.

    Parameters:
        cities_count (int): The number of cities in the dataset.
        pop_size (int): The number of tours to generate.

    Returns:
        list of lists: A population of TSP tours. Each tour is a list of city indices representing
                       a cycle (starts and ends at the same city).
    """
    population = []
    for _ in range(pop_size):
        tour = list(range(cities_count))
        rng.shuffle(tour)  # Use rng for reproducibility
        tour.append(tour[0])  # Ensure the tour is a cycle
        population.append(tour)
    return population

### Selection Mechanism: Tournament Selection

The `tournament_selection` function implements the tournament selection strategy, which selects the best individual out of a randomly chosen subset of the population. This method maintains selective pressure towards better solutions.

**Parameters:**
- `population` (`list of lists`): The population of tours.
- `dist_matrix` (`np.ndarray`): The distance matrix for the cities.
- `k` (`int`): The number of individuals to sample for the tournament. Default is 15.

**Returns:**
- `list`: The selected tour (winner of the tournament).

In [6]:
def tournament_selection(population, dist_matrix, k=15):
    """
    Select a tour from the population using tournament selection.

    Parameters:
        population (list of lists): The population of tours.
        dist_matrix (np.ndarray): The distance matrix for the cities.
        k (int): The number of individuals to sample for the tournament. Default is 15.

    Returns:
        list: The selected tour (winner of the tournament).
    """
    selected = rng.choice(population, k, replace=False)
    selected_sorted = sorted(selected, key=lambda tsp: tsp_cost(tsp, dist_matrix))
    return selected_sorted[0]

### Crossover Mechanism: Order Crossover (OX)

The `order_crossover` function performs Order Crossover (OX) between two parent tours to produce an offspring. This method preserves the relative order of cities, maintaining valid tours.

**Parameters:**
- `parent1` (`list`): The first parent tour.
- `parent2` (`list`): The second parent tour.

**Returns:**
- `list`: The offspring tour generated by combining segments of the two parents.

Ensures that the offspring is a cycle by setting the last element to match the first.

In [7]:
def order_crossover(parent1, parent2):
    """
    Perform order crossover (OX) between two parent tours to produce an offspring tour.

    Parameters:
        parent1 (list): The first parent tour.
        parent2 (list): The second parent tour.

    Returns:
        list: The offspring tour generated by combining segments of the two parents.
    """
    size = len(parent1) - 1  # Exclude the last city since it's a cycle
    start, end = sorted(rng.choice(range(size), 2, replace=False))
    child = [-1] * size
    child[start:end + 1] = parent1[start:end + 1]
    
    ptr = 0
    for city in parent2:
        if city not in child:
            while child[ptr] != -1:
                ptr += 1
            child[ptr] = city
    child.append(child[0])  # Ensure the child tour is a cycle
    return child

### Mutation Mechanism: Inversion Mutation

The `inversion_mutation` function applies inversion mutation to a TSP tour. It selects a random segment of the tour and reverses its order, introducing variability while maintaining valid tours.

**Parameters:**
- `tour` (`list`): The tour to mutate.

**Returns:**
- `list`: The mutated tour.

Ensures the tour remains a cycle after mutation.

In [8]:
def inversion_mutation(tour):
    """
    Apply inversion mutation to a TSP tour.

    Parameters:
        tour (list): The tour to mutate.

    Returns:
        list: The mutated tour.

    Selects a random segment of the tour and reverses its order. Ensures the tour remains a cycle.
    """
    size = len(tour) - 1  # Exclude the last city since it's a cycle
    start, end = sorted(rng.choice(range(size), 2, replace=False))
    mutated_segment = list(reversed(tour[start:end + 1]))
    tour[start:end + 1] = mutated_segment
    if tour[-1] != tour[0]:  # Ensure the tour is a cycle
        tour[-1] = tour[0]
    return tour

## Genetic Algorithm Implementation

The Genetic Algorithm (GA) is an evolutionary algorithm inspired by natural selection. It iteratively evolves a population of candidate solutions towards better solutions. In the context of TSP, each individual in the population represents a possible tour.

### Key Components:
- **Initialization**: Generate an initial population of random tours.
- **Selection**: Select parent tours based on their fitness (tour cost).
- **Crossover**: Combine parent tours to produce offspring.
- **Mutation**: Introduce random variations to offspring tours.
- **Elitism**: Preserve a subset of the best tours across generations.
- **Termination**: Repeat the process for a predefined number of generations or until convergence.

### Genetic Algorithm Function

In [9]:
def ea_tsp(cities, pop_size, generations):
    """
    Solve the TSP using a genetic algorithm.

    Parameters:
        cities (pd.DataFrame): DataFrame containing city information.
        pop_size (int): The number of individuals (tours) in the population.
        generations (int): The number of generations to evolve the population.

    Returns:
        tuple: The best tour found, its associated cost, the generation when it was first found,
               and the cost progression over generations.
    """
    dist_matrix = create_distance_matrix(cities)
    population = initialize_population(len(cities), pop_size * 2)  # Increase initial population
    
    best_solution = min(population, key=lambda tsp: tsp_cost(tsp, dist_matrix))
    best_cost = tsp_cost(best_solution, dist_matrix)
    first_generation_ea = 0  # Initialize to 0 for the first generation
    cost_progression = [best_cost]  # To track cost over generations
    
    total_time = 0  # Total elapsed time
    start_time = time.time()  # Start time
    
    for generation in range(1, generations + 1):
        gen_start_time = time.time()
        
        new_population = []
        
        # Elitism: Keep the top 5% of the current generation
        elite_count = max(1, int(0.05 * pop_size))  # Ensure at least one elite
        elite_individuals = sorted(population, key=lambda tsp: tsp_cost(tsp, dist_matrix))[:elite_count]
        new_population.extend(elite_individuals)
        
        # Fill the rest of the population with crossover and mutation
        while len(new_population) < pop_size:
            # Tournament selection with higher pressure
            parent1 = tournament_selection(population, dist_matrix, k=10)
            parent2 = tournament_selection(population, dist_matrix, k=10)
            
            # Apply crossover with a higher probability
            if rng.random() < 0.9:
                child1 = order_crossover(parent1, parent2)
                child2 = order_crossover(parent2, parent1)
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            
            # Apply mutation with a dynamically decreasing probability
            mutation_rate = max(0.01, 0.1 - (0.1 * generation / generations))
            if rng.random() < mutation_rate:
                child1 = inversion_mutation(child1)
            if rng.random() < mutation_rate:
                child2 = inversion_mutation(child2)
            
            new_population.extend([child1, child2])
        
        # Trim excess individuals if population exceeds pop_size
        population = new_population[:pop_size]
        
        # Update best solution
        current_best = min(population, key=lambda tsp: tsp_cost(tsp, dist_matrix))
        current_cost = tsp_cost(current_best, dist_matrix)
        cost_progression.append(current_cost)
        
        if current_cost < best_cost:
            best_solution, best_cost = current_best, current_cost
            first_generation_ea = generation  # Update the first generation when new best is found
            logging.info(f"Generation {generation}: New best cost = {best_cost:.2f} km")
        
        # Optional: Log progress every 50 generations
        if generation % 50 == 0:
            elapsed_time = time.time() - start_time
            average_time_per_gen = elapsed_time / generation
            remaining_gens = generations - generation
            estimated_remaining_time = average_time_per_gen * remaining_gens
            logging.info(f"Generation {generation}: Current best cost = {best_cost:.2f} km")
            logging.info(f"Elapsed Time: {elapsed_time/60:.2f} minutes | Estimated Remaining Time: {estimated_remaining_time/60:.2f} minutes")

    total_time = time.time() - start_time
    logging.info(f"Best solution found with cost = {best_cost:.2f} km in {total_time:.2f} seconds at generation {first_generation_ea}")
    return best_solution, best_cost, first_generation_ea, cost_progression

### Running the Genetic Algorithm on Multiple Datasets

The `run_multiple_datasets_ea` function executes the Genetic Algorithm on multiple city datasets and records the results in a CSV file. This allows for comparative analysis across different geographical regions.

**Parameters:**
- `filenames` (`list of str`): List of file paths for the city datasets.
- `output_file` (`str`): Path to the output CSV file.
- `generations` (`int`): The number of generations to evolve the population for each dataset.

**Functionality:**
- Loads each dataset.
- Determines an adaptive population size based on the number of cities.
- Runs the Genetic Algorithm.
- Records the best tour and its cost.
- Handles exceptions and logs errors if any occur during processing.

In [10]:
def run_multiple_datasets_ea(filenames, output_file, generations=25):
    """
    Run the genetic algorithm on multiple datasets and save the results to a CSV file.

    Parameters:
        filenames (list of str): List of file paths for the city datasets.
        output_file (str): Path to the output CSV file.
        generations (int): The number of generations to evolve the population for each dataset.

    Returns:
        dict: Progression data for plotting.
    """
    results = []
    ea_progressions = {}  # To store progression data for plotting
    
    for filename in filenames:
        try:
            # Load cities data
            cities = pd.read_csv(filename, header=None, names=['name', 'lat', 'lon'])
            cities_count = len(cities)
            population_size = adaptive_population(cities_count)
            
            logging.info(f"Processing {filename} with {cities_count} cities. Population size set to {population_size}.")
            
            # Run genetic algorithm
            best_tour, best_cost, first_generation_ea, cost_progression = ea_tsp(cities, pop_size=population_size, generations=generations)
            
            # Append results
            results.append({
                'filename': filename,
                'num_cities': cities_count,
                'best_cost_km': round(best_cost, 2),
                'first_generation_ea': first_generation_ea,
                'best_tour': best_tour
            })
            
            # Store progression data
            ea_progressions[filename] = cost_progression
            
            logging.info(f"Completed TSP for {filename}. Best cost: {best_cost:.2f} km at generation {first_generation_ea}")
        
        except Exception as e:
            logging.error(f"An error occurred while processing {filename}: {e}")
            results.append({
                'filename': filename,
                'num_cities': len(cities) if 'cities' in locals() else 'N/A',
                'best_cost_km': 'Error',
                'first_generation_ea': 'Error',
                'best_tour': 'Error'
            })
    
    # Save results to CSV
    try:
        with open(output_file, mode='w', newline='') as file:
            fieldnames = ['filename', 'num_cities', 'best_cost_km', 'first_generation_ea', 'best_tour']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for result in results:
                writer.writerow(result)
        logging.info(f"EA Results successfully saved to {output_file}")
    except Exception as e:
        logging.error(f"Failed to save EA results to {output_file}: {e}")
    
    return ea_progressions  # Return progression data for plotting

## Greedy Algorithm Implementation

The Greedy Algorithm provides a heuristic-based solution to the TSP by iteratively selecting the nearest unvisited city. While it doesn't guarantee an optimal solution, it offers a quick approximation suitable for larger datasets.

### Greedy Algorithm Function

In [11]:
def greedy_tsp(cities):
    """
    Solve the TSP using a greedy approach.

    Parameters:
        cities (pd.DataFrame): DataFrame with columns 'name', 'lat', 'lon'.

    Returns:
        tuple: The best tour found, its associated cost, the step when the tour was completed,
               and the cost progression over steps.
    """
    dist_matrix = create_distance_matrix(cities)
    num_cities = len(cities)
    
    # Start from the first city
    tour = [0]
    visited = set(tour)
    cost_progression = []  # To track cost after each step
    
    while len(tour) < num_cities:
        last = tour[-1]
        # Find the nearest unvisited city
        next_city = min(
            (i for i in range(num_cities) if i not in visited),
            key=lambda x: dist_matrix[last, x]
        )
        tour.append(next_city)
        visited.add(next_city)
        current_cost = tsp_cost(tour, dist_matrix)
        cost_progression.append(current_cost)
        logging.debug(
            f"step: {cities.at[last,'name']} -> {cities.at[next_city,'name']} ({dist_matrix[last, next_city]:.2f}km), Current Cost: {current_cost:.2f}km"
        )
    
    # Complete the tour by returning to the starting city
    tour.append(tour[0])
    total_cost = tsp_cost(tour, dist_matrix)
    cost_progression.append(total_cost)
    logging.debug(
        f"step: {cities.at[tour[-2],'name']} -> {cities.at[tour[0],'name']} ({dist_matrix[tour[-2], tour[0]]:.2f}km), Total Cost: {total_cost:.2f}km"
    )
    
    logging.info(f"result: Found a path of {len(tour)-1} steps, total length {total_cost:.2f}km")
    first_step_greedy = len(tour) - 1  # The final step completes the tour
    
    return tour, total_cost, first_step_greedy, cost_progression

### Running the Greedy Algorithm on Multiple Datasets

The `run_multiple_datasets_greedy` function executes the Greedy Algorithm on multiple city datasets and records the results in a CSV file. This facilitates a performance comparison between the Genetic and Greedy approaches.

**Parameters:**
- `filenames` (`list of str`): List of file paths for the city datasets.
- `output_file` (`str`): Path to the output CSV file.

**Functionality:**
- Loads each dataset.
- Runs the Greedy Algorithm.
- Records the best tour and its cost.
- Handles exceptions and logs errors if any occur during processing.

In [12]:
def run_multiple_datasets_greedy(filenames, output_file):
    """
    Run the greedy TSP algorithm on multiple datasets and save the results to a CSV file.

    Parameters:
        filenames (list of str): List of file paths for the city datasets.
        output_file (str): Path to the output CSV file.

    Returns:
        dict: Progression data for plotting.
    """
    results = []
    greedy_progressions = {}  # To store progression data for plotting
    
    for filename in filenames:
        try:
            # Load cities data
            cities = pd.read_csv(filename, header=None, names=['name', 'lat', 'lon'])
            cities_count = len(cities)
            
            logging.info(f"Processing {filename} with {cities_count} cities using greedy algorithm.")
            
            # Run greedy TSP
            best_tour, best_cost, first_step_greedy, cost_progression = greedy_tsp(cities)
            
            # Append results
            results.append({
                'filename': filename,
                'num_cities': cities_count,
                'best_cost_km': round(best_cost, 2),
                'first_step_greedy': first_step_greedy,
                'best_tour': best_tour
            })
            
            # Store progression data
            greedy_progressions[filename] = cost_progression
            
            logging.info(f"Completed TSP for {filename} using greedy approach. Best cost: {best_cost:.2f} km at step {first_step_greedy}")
        
        except Exception as e:
            logging.error(f"An error occurred while processing {filename} with greedy algorithm: {e}")
            results.append({
                'filename': filename,
                'num_cities': len(cities) if 'cities' in locals() else 'N/A',
                'best_cost_km': 'Error',
                'first_step_greedy': 'Error',
                'best_tour': 'Error'
            })
    
    # Save results to CSV
    try:
        with open(output_file, mode='w', newline='') as file:
            fieldnames = ['filename', 'num_cities', 'best_cost_km', 'first_step_greedy', 'best_tour']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for result in results:
                writer.writerow(result)
        logging.info(f"Greedy Results successfully saved to {output_file}")
    except Exception as e:
        logging.error(f"Failed to save Greedy results to {output_file}: {e}")
    
    return greedy_progressions  # Return progression data for plotting

## Running Experiments

In this section, we define the datasets to be analyzed and execute both the Genetic and Greedy algorithms on these datasets. The results are saved to respective CSV files for further analysis.

**Datasets Included:**
- `italy.csv`
- `vanuatu.csv`
- `russia.csv`
- `us.csv`
- `china.csv`

**Note:** Ensure that the datasets are located in the specified `cities/` directory or adjust the file paths accordingly.

In [13]:
# List of files to analyze and output files
filenames = [
    "cities/italy.csv",
    "cities/vanuatu.csv",
    "cities/russia.csv",
    "cities/us.csv",
    "cities/china.csv"
]

# Execute the calculations for EA
output_file_ea = "tsp_results_ea.csv"
ea_progressions = run_multiple_datasets_ea(filenames, output_file_ea, generations=1000)
print(f"EA Results saved to {output_file_ea}")

# Execute the calculations for Greedy
output_file_greedy = "tsp_results_greedy.csv"
greedy_progressions = run_multiple_datasets_greedy(filenames, output_file_greedy)
print(f"Greedy Results saved to {output_file_greedy}")

INFO:root:Processing cities/italy.csv with 46 cities. Population size set to 100.
INFO:root:Generation 1: New best cost = 14890.41 km
INFO:root:Generation 2: New best cost = 14642.97 km
INFO:root:Generation 3: New best cost = 13659.68 km
INFO:root:Generation 4: New best cost = 13454.59 km
INFO:root:Generation 5: New best cost = 13045.51 km
INFO:root:Generation 6: New best cost = 12822.15 km
INFO:root:Generation 7: New best cost = 12427.94 km
INFO:root:Generation 9: New best cost = 11872.47 km
INFO:root:Generation 10: New best cost = 11654.84 km
INFO:root:Generation 11: New best cost = 11304.49 km
INFO:root:Generation 12: New best cost = 11273.46 km
INFO:root:Generation 13: New best cost = 10852.27 km
INFO:root:Generation 14: New best cost = 10158.20 km
INFO:root:Generation 15: New best cost = 9960.86 km
INFO:root:Generation 16: New best cost = 9736.65 km
INFO:root:Generation 18: New best cost = 9691.17 km
INFO:root:Generation 19: New best cost = 9419.34 km
INFO:root:Generation 20: New 

EA Results saved to tsp_results_ea.csv


INFO:root:result: Found a path of 167 steps, total length 42334.16km
INFO:root:Completed TSP for cities/russia.csv using greedy approach. Best cost: 42334.16 km at step 167
INFO:root:Processing cities/us.csv with 326 cities using greedy algorithm.
INFO:root:result: Found a path of 326 steps, total length 48050.03km
INFO:root:Completed TSP for cities/us.csv using greedy approach. Best cost: 48050.03 km at step 326
INFO:root:Processing cities/china.csv with 726 cities using greedy algorithm.
INFO:root:result: Found a path of 726 steps, total length 63962.92km
INFO:root:Completed TSP for cities/china.csv using greedy approach. Best cost: 63962.92 km at step 726
INFO:root:Greedy Results successfully saved to tsp_results_greedy.csv


Greedy Results saved to tsp_results_greedy.csv


**Note:** The number of generations for the Genetic Algorithm has been set to `1000` as per your original script, which may take several hours to complete depending on the dataset size and computational resources.

## Results and Analysis

After running the experiments, the results from both the Genetic and Greedy algorithms are saved in `tsp_results_ea.csv` and `tsp_results_greedy.csv` respectively. These results include the filename, number of cities, best cost in kilometers, and the best tour found.

**Sample Output:**
```plaintext
EA Results saved to tsp_results_ea.csv
Greedy Results saved to tsp_results_greedy.csv
```

The results provide insights into the performance of each algorithm across different datasets. By analyzing the best costs and tours, we can evaluate the efficiency and effectiveness of the Genetic and Greedy approaches in solving the TSP.

## Plotting Progressions

To visualize the performance of both algorithms over their execution, we can plot the cost progression across generations (for the Genetic Algorithm) and steps (for the Greedy Algorithm). This helps in understanding how each algorithm converges towards a solution.

### Plotting EA Progression

In [14]:
def plot_ea_progress(ea_progressions, output_dir="plots/ea"):
    """
    Plot the evolution of the EA algorithm's cost over generations for each dataset.

    Parameters:
        ea_progressions (dict): Dictionary where keys are filenames and values are lists of costs per generation.
        output_dir (str): Directory where the plots will be saved.
    """
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename, progression in ea_progressions.items():
        plt.figure(figsize=(10, 6))
        plt.plot(progression, label='Best Cost')
        plt.title(f'EA Cost Progression for {filename}')
        plt.xlabel('Generation')
        plt.ylabel('Cost (km)')
        plt.legend()
        plt.grid(True)
        plot_filename = os.path.join(output_dir, f"{os.path.basename(filename)}_ea_progression.png")
        plt.savefig(plot_filename)
        plt.close()
        logging.info(f"EA progression plot saved to {plot_filename}")

### Plotting Greedy Progression

In [15]:
def plot_greedy_progress(greedy_progressions, output_dir="plots/greedy"):
    """
    Plot the evolution of the Greedy algorithm's cost over steps for each dataset.

    Parameters:
        greedy_progressions (dict): Dictionary where keys are filenames and values are lists of costs per step.
        output_dir (str): Directory where the plots will be saved.
    """
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename, progression in greedy_progressions.items():
        plt.figure(figsize=(10, 6))
        plt.plot(progression, label='Cumulative Cost')
        plt.title(f'Greedy Cost Progression for {filename}')
        plt.xlabel('Step')
        plt.ylabel('Cost (km)')
        plt.legend()
        plt.grid(True)
        plot_filename = os.path.join(output_dir, f"{os.path.basename(filename)}_greedy_progression.png")
        plt.savefig(plot_filename)
        plt.close()
        logging.info(f"Greedy progression plot saved to {plot_filename}")

### Executing the Plotting Functions

In [16]:
# Plot the progression of EA
plot_ea_progress(ea_progressions)

# Plot the progression of Greedy
plot_greedy_progress(greedy_progressions)

print("Plotting completed. Check the 'plots' directory for the generated graphs.")

INFO:root:EA progression plot saved to plots/ea/italy.csv_ea_progression.png
INFO:root:EA progression plot saved to plots/ea/vanuatu.csv_ea_progression.png
INFO:root:EA progression plot saved to plots/ea/russia.csv_ea_progression.png
INFO:root:EA progression plot saved to plots/ea/us.csv_ea_progression.png
INFO:root:EA progression plot saved to plots/ea/china.csv_ea_progression.png
INFO:root:Greedy progression plot saved to plots/greedy/italy.csv_greedy_progression.png
INFO:root:Greedy progression plot saved to plots/greedy/vanuatu.csv_greedy_progression.png
INFO:root:Greedy progression plot saved to plots/greedy/russia.csv_greedy_progression.png
INFO:root:Greedy progression plot saved to plots/greedy/us.csv_greedy_progression.png
INFO:root:Greedy progression plot saved to plots/greedy/china.csv_greedy_progression.png


Plotting completed. Check the 'plots' directory for the generated graphs.
